# Structure of PAV core counterexamples for $k = 8$

This notebook contains code for solving the linear programs appearing in the proof of Lemma 4.4 (about committee size $k = 8$). The program also stores fractional dual solutions in a file. The correctness of these dual solutions can be verified using the code in the notebook `duals-counterexample-k8-structure.ipynb`.

In [1]:
from gurobipy import Env, Model, GRB, quicksum
import itertools
import os
from functools import cache
import json
import pickle
from tqdm import tqdm
from fractions import Fraction
from utility_functions import powerset, show, harmonic_number, utility, generate_wlog_partition, is_wlog

The first step in the proof is to establish that the only possible deviation to a PAV committee for $k = 8$ is one with two committee members and two non-committee members. This can be established via the argument for Theorem 4.1 in the paper, together with the code in the notebook `check-inequality.ipynb` evaluated for $k = 8$.

Next, the proof argues that we may restrict attention to the set of candidates $C = W \cup T$, which is of size 10. We perform these initializations in the code below.

In [2]:
num_alts = 10
k = 8
A = list(range(num_alts))
ballots = sorted(powerset(A))
committees = list(itertools.combinations(A, k))
pav_committee = tuple(A[:k])
T = (0,1,8,9)

The function `make_model` produces a gurobi `Model` that contains the constraints labeled (4) in the paper. In particular, the model has a variable `freq[ballot]` denoting the fraction of the profile made up by the ballot. It then has constraints saying that $W$ is a PAV committee (i.e., swap-stable), and that $T$ is a successful deviation.

In [3]:
def make_model(integral=False):
    m = Model()
    m.params.OutputFlag = 0
    freq = {ballot: m.addVar() for ballot in ballots}
    m.addConstr(quicksum(freq[ballot] for ballot in ballots) == 1, "alpha")
    
    score = {committee: quicksum(freq[ballot] * harmonic_number(utility(ballot, committee)) for ballot in ballots) for committee in committees}
    opt_score = score[pav_committee]

    swaps = [(x,y) for x in pav_committee for y in set(A) - set(pav_committee)]
    W_xy = {(x,y) : tuple((set(pav_committee) - {x}) | {y}) for x,y in swaps}

    for x, y in swaps:
        m.addConstr(score[W_xy[(x,y)]] <= opt_score, f"beta_{x}_{y}")

    ballots_preferring_T = set(ballot for ballot in ballots if utility(ballot, T) > utility(ballot, pav_committee))
    # constraint negated to get the right sign of the dual variable
    m.addConstr(-quicksum(freq[ballot] for ballot in ballots_preferring_T) <= -len(T) / k, "gamma")
    return m, freq

The following code block contains a function that extracts the dual solution computed by gurobi. It then converts them to exact rational numbers (by going through a "library" of rational numbers with small denominators and looking for one that is very close to the float from the gurobi solution). The dual solutions are then stored in a file.

In [4]:
class FractionEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Fraction):
            # Serialize the fraction as a string representation "numerator/denominator"
            return f"{o.numerator}/{o.denominator}"
        return super(FractionEncoder, self).default(o)

# fractions library
fractions = [0]
for denom in list(range(1, 12)) + [9]:
    for num in range(1, max(4*denom, 10)):
        fractions.append(Fraction(num, denom))
fractions += [Fraction(15, 14), Fraction(56, 39), Fraction(112, 39), Fraction(1, 6), Fraction(11, 6), Fraction(30, 29), Fraction(60, 29), Fraction(3, 7), Fraction(4, 7), Fraction(5, 8), Fraction(5, 7), Fraction(6, 7), Fraction(20, 23), Fraction(7, 8), Fraction(8, 7), Fraction(10, 7), Fraction(20, 13), Fraction(12, 7), Fraction(40, 23), Fraction(30, 17), Fraction(40, 13), Fraction(60, 17), Fraction(10, 1), Fraction(12, 1), Fraction(14, 1), Fraction(11, 2)]
fractions += [-frac for frac in fractions]

pkl_filename = "dual_values_k8.pkl"
json_filename = "dual_values_k8.json"
if not os.path.exists(pkl_filename):
    pickle.dump({}, open(pkl_filename, "wb"))
not_found = set()
def save_dual(m, name=None):
    if m.ModelSense == 1: # maximization
        m.setObjective(-m.getObjective(), GRB.MINIMIZE)
        m.optimize()
    dual_values = {}
    for constr in m.getConstrs():
        value = constr.Pi # dual value
        if not any(abs(frac - value) < 1e-6 for frac in fractions):
            print(f"not found in fraction library: {value}")
            not_found.add(value)
        frac = min(fractions, key=lambda x: abs(x - value))
        dual_values[constr.ConstrName] = frac
    if name:
        saved_values = pickle.load(open(pkl_filename, "rb"))
        saved_values[name] = dual_values
        pickle.dump(saved_values, open(pkl_filename, "wb"))
        json.dump(saved_values, open(json_filename, "w"), indent=2, cls=FractionEncoder)

## First program type: ballots in the blocking coalition

The first batch of programs concerns the frequency with which ballots supporting $T$ appear in the profile, with both $\{0, 1, 8\}$ and $\{0, 1, 9\}$ appearing with frequency $1/4$ (after restricting the candidate set to $W \cup T$).

The code more generally looks through all possible ballots that prefer $T$ over $W$, and finds that no other such ballots can appear in the profile.

In [5]:
T = (0,1,8,9)
ballots_preferring_T = set(ballot for ballot in ballots if utility(ballot, T) > utility(ballot, pav_committee))
for ballot in ballots_preferring_T:
    m, freq = make_model(integral=False)
    m.setObjective(freq[ballot], GRB.MAXIMIZE)
    m.optimize()
    if m.status == GRB.OPTIMAL and m.objVal > 0.00001:
        save_dual(m, name=f"max_freq_{show(ballot)}")
        max_weight = m.objVal
        m.setObjective(-freq[ballot], GRB.MAXIMIZE)
        m.optimize()
        min_weight = m.objVal
        print(f"Blocking coalition can include {ballot} with weight between {min_weight} and {max_weight}")
        save_dual(m, name=f"min_freq_{show(ballot)}")


Set parameter Username
Set parameter LicenseID to value 2591460
Academic license - for non-commercial use only - expires 2025-11-26
Blocking coalition can include (0, 1, 9) with weight between -0.25000000000000006 and 0.25000000000000006
Blocking coalition can include (0, 1, 8) with weight between -0.25000000000000006 and 0.25000000000000006


## Second program type: ballots outside the blocking coalition

The second batch of programs verifies that, besides $\{0, 1, 8\}$ and $\{0, 1, 9\}$, all other ballots are disjoint from $T$. We do this by considering each ballot that intersects $T$ and verifying that its maximum frequency in the profile is $0$.

The code below verifies this only for the ballots that are "canonical" to speed up the computation.

In [6]:
ballots_intersecting_T = [ballot for ballot in ballots if any(x in T for x in ballot)]
partition = generate_wlog_partition(A, [T, pav_committee])
ballots_intersecting_T = [ballot for ballot in ballots_intersecting_T if is_wlog(ballot, partition)]
for ballot in tqdm(ballots_intersecting_T):
    if ballot == (0, 1, 8):
        continue # handled above
    m, freq = make_model(integral=False)
    m.setObjective(freq[ballot], GRB.MAXIMIZE)
    m.optimize()
    assert m.objVal < 0.00001
    if m.status == GRB.OPTIMAL:
        save_dual(m, name=f"max_freq_{show(ballot)}")

100%|██████████| 55/55 [00:03<00:00, 17.04it/s]


## Third program type: change of PAV score when removing a candidate

The third program computes by how much the PAV score of the committee $W$ changes if any one member of $W \setminus \{0,1\}$ is removed. Without loss of generality, we only need to consider removing $x = 2$. The answer is $-1/12$.

In [7]:
def pav_score(freq, committee):
    return quicksum(freq[ballot] * harmonic_number(utility(ballot, committee)) for ballot in ballots)

x = 2 # without loss of generality
m, freq = make_model()
pav_committee_without_x = tuple(a for a in pav_committee if a != x)
marginal = pav_score(freq, pav_committee_without_x) - pav_score(freq, pav_committee)
m.setObjective(marginal, GRB.MAXIMIZE)
m.optimize()
save_dual(m, name=f"max_marginal_{x}")
max_marginal = m.objVal
m.setObjective(-marginal, GRB.MAXIMIZE)
m.optimize()
save_dual(m, name=f"min_marginal_{x}")
min_marginal = m.objVal
print(f"Removing {x} from the PAV committee can decrease the score by between {min_marginal} and {-max_marginal}")

Removing 2 from the PAV committee can decrease the score by between 0.08333333333333343 and 0.08333333333333331
